In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [ ]:
sampling_rate = 44100

In [ ]:
scale_file = "data\\test\\segment_1.wav"

In [ ]:
ipd.Audio(scale_file, rate = sampling_rate)

In [ ]:
scale,sr = librosa.load(scale_file)

In [ ]:
scale

In [ ]:
sr

In [ ]:
d = librosa.stft(scale)
scale_ft = librosa.amplitude_to_db(np.abs(d), ref=np.max)
scale_ft.shape

In [ ]:
#plot the transofrmed audio
fig, ax = plt.subplots(figsize = (10,5))
img = librosa.display.specshow(scale_ft,
                               x_axis='time', 
                               y_axis='log',ax=ax)
ax.set_title("Spectogram example", fontsize=20)
fig.colorbar(img,ax=ax, format=f'%0.2f')
plt.show()

In [ ]:
filter_banks = librosa.filters.mel(n_fft=2048, sr=22050, n_mels=10)

In [ ]:
filter_banks.shape

In [ ]:
plt.figure(figsize=(25,10))
librosa.display.specshow(filter_banks, sr=sr, x_axis="linear")

plt.colorbar(format="%+2.f")
plt.show()

In [ ]:
mel_spectrogram = librosa.feature.melspectrogram(y=scale, sr=sr, n_fft=2048, hop_length=512, n_mels=10)

In [ ]:
mel_spectrogram.shape

In [ ]:
log_mel_spectogram = librosa.power_to_db(mel_spectrogram)

In [ ]:
log_mel_spectogram.shape

In [ ]:
plt.figure(figsize=(25,10))
librosa.display.specshow(log_mel_spectogram, x_axis = "time", y_axis="mel", sr=sr)

plt.colorbar(format="%+2.f")
plt.show()


In [ ]:
import torchaudio
import torch
import matplotlib.pyplot as plt
import numpy as np

# Load the WAV file
waveform, sample_rate = torchaudio.load('data/scale.wav')

# Convert to mono if stereo (many models expect mono)
if waveform.shape[0] == 2:  # Stereo
    waveform = torch.mean(waveform, dim=0, keepdim=True)  # Average channels

# Compute mel spectrogram with common parameters
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_mels=128,  # Standard for audio classification
    n_fft=1024,  # FFT window size
    hop_length=512  # Hop size for time resolution
)
mel_spectrogram = mel_transform(waveform)

# Convert to decibel scale for better visualization and sometimes model input
db_transform = torchaudio.transforms.AmplitudeToDB()
mel_spectrogram_db = db_transform(mel_spectrogram)

# Save as numerical data (e.g., for model input)
mel_spectrogram_np = mel_spectrogram_db.squeeze().numpy()  # Remove channel dim if mono
np.save('mel_spectrogram.npy', mel_spectrogram_np)

# Alternatively, save as PyTorch tensor
# torch.save(mel_spectrogram_db, 'mel_spectrogram.pt')

# Save as image for visualization
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram_db.squeeze().numpy(), aspect='auto', origin='lower', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram (dB)')
plt.tight_layout()
plt.savefig('mel_spectrogram.png')
plt.show()

In [ ]:
import os
import torch
import torchaudio

for filename in os.listdir('data/'):
    if filename.endswith('.wav'):
        waveform, sample_rate = torchaudio.load(os.path.join('data/', filename))
        if waveform.shape[0] == 2:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        mel_spectrogram = mel_transform(waveform)
        mel_spectrogram_db = db_transform(mel_spectrogram)
        mel_spectrogram_np = mel_spectrogram_db.squeeze().numpy()
        np.save(f'mel_spectrogram_{filename[:-4]}.npy', mel_spectrogram_np)
        plt.figure(figsize=(10, 4))
        plt.imshow(mel_spectrogram_db.squeeze().numpy(), aspect='auto', origin='lower', cmap='viridis')
        plt.colorbar(format='%+2.0f dB')
        plt.title(f'Mel Spectrogram (dB) - {filename}')
        plt.tight_layout()
        plt.savefig(f'mel_spectrogram_{filename[:-4]}.png')
        plt.close()  # Close figure to avoid memory issues

Steps
Load Audio: Use librosa.load to get the audio samples and sampling rate.
Apply A-Weighting: Filter the audio to mimic human hearing perception.
Compute Leq: Calculate the mean squared pressure and convert to dBA.
Handle Output: Return or display the Leq value.

In [ ]:
import librosa
import numpy as np
import IPython.display as ipd

# File path
audio_file = "data\\noiseData\\Around Arya School(1°16_32_ S 36°49_29_ E).wav"

# Load audio
y, sr = librosa.load(audio_file, sr=None)  # y: audio samples, sr: sampling rate

# Apply A-weighting (approximation using librosa's perceptual weighting)
# librosa doesn't have direct A-weighting, so we use a simplified approach
# For precise A-weighting, you may need scipy or custom filters
def apply_a_weighting(signal, sr):
    # Approximate A-weighting by emphasizing 1-6 kHz (simplified for demonstration)
    # For production, use scipy.signal or a dedicated filter (e.g., A-weighting coefficients)
    freqs = np.fft.rfftfreq(len(signal), d=1/sr)
    a_weights = np.ones_like(freqs)
    # Simplified A-weighting curve (dB adjustment)
    for i, f in enumerate(freqs):
        if f < 20 or f > 20000:
            a_weights[i] = 0  # Attenuate outside human hearing
        elif 1000 <= f <= 6000:
            a_weights[i] = 1.0  # Emphasize mid frequencies
        else:
            a_weights[i] = 0.8  # Slight attenuation elsewhere
    # Apply to frequency domain
    spectrum = np.fft.rfft(signal)
    spectrum_weighted = spectrum * a_weights
    signal_weighted = np.fft.irfft(spectrum_weighted, n=len(signal))
    return signal_weighted

# Apply A-weighting
y_a_weighted = apply_a_weighting(y, sr)

# Compute Leq
p_ref = 2e-5  # Reference pressure (Pa)
#mean_square = np.mean(y_a_weighted**2)  # Mean squared pressure
#leq = 10 * np.log10(mean_square / (p_ref**2))

mean_square = np.mean(y_a_weighted**2)  # From A-weighting code
leq = 10 * np.log10(mean_square / (p_ref)**2)
print(f"Leq (dBA): {leq:.2f}")

print(f"Leq (dBA): {leq:.2f}")

# Optional: Play audio to verify
ipd.Audio(audio_file)  # No rate needed for file path

Different reference pressure below

In [ ]:
import librosa
import numpy as np
import IPython.display as ipd
from scipy.signal import butter, lfilter

# File path
audio_file = "data\\noiseData\\Around Arya School(1°16_32_ S 36°49_29_ E).wav"

# Load audio
y, sr = librosa.load(audio_file, sr=None)  # y: audio samples, sr: sampling rate

# Noise Reduction
def spectral_subtraction(audio, sr, noise_start=0.0, noise_end=1.0):
    noise_start_sample = int(noise_start * sr)
    noise_end_sample = int(noise_end * sr)
    noise = audio[noise_start_sample:noise_end_sample]
    noise_spec = np.abs(librosa.stft(noise, n_fft=2048, hop_length=512))
    noise_mean = np.mean(noise_spec, axis=1, keepdims=True)
    audio_spec = np.abs(librosa.stft(audio, n_fft=2048, hop_length=512))
    cleaned_spec = np.maximum(audio_spec - noise_mean, 0)
    return librosa.istft(cleaned_spec, hop_length=512, length=len(audio))

y_cleaned = spectral_subtraction(y, sr, noise_start=0.0, noise_end=1.0)

# Low-pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def lowpass_filter(data, cutoff_freq, fs, order=5):
    b, a = butter_lowpass(cutoff_freq, fs, order=order)
    y = lfilter(b, a, data)
    return y

y_filtered = lowpass_filter(y_cleaned, cutoff_freq=4000, fs=sr)

# Apply A-weighting
def apply_a_weighting(signal, sr):
    freqs = np.fft.rfftfreq(len(signal), d=1/sr)
    a_weights = np.ones_like(freqs)
    for i, f in enumerate(freqs):
        if f < 20 or f > 20000:
            a_weights[i] = 0
        elif 1000 <= f <= 6000:
            a_weights[i] = 1.0
        else:
            a_weights[i] = 0.8
    spectrum = np.fft.rfft(signal)
    spectrum_weighted = spectrum * a_weights
    return np.fft.irfft(spectrum_weighted, n=len(signal))

y_a_weighted = apply_a_weighting(y_filtered, sr)

# Model reference pressure and calibrate signal
p_ref = 2e-5  # Standard reference pressure (Pa)
calibration_spl = 94.0  # Calibration level (dB)
calibration_pressure = 2e-5 * (10 ** (calibration_spl / 20))  # 1 Pa at 94 dB

# Adjust signal amplitude based on calibration
# Assume max amplitude of y corresponds to 94 dB (1 Pa), scale accordingly
max_amplitude = np.max(np.abs(y))
if max_amplitude > 0:
    scale_factor = calibration_pressure / max_amplitude  # Scale to 1 Pa
    y_a_weighted_calibrated = y_a_weighted * scale_factor
else:
    y_a_weighted_calibrated = y_a_weighted  # Avoid division by zero

# Compute Leq
mean_square = np.mean(y_a_weighted_calibrated**2)
leq = 10 * np.log10(mean_square / (p_ref**2))

print(f"Modeled Reference Pressure: {p_ref:.6e} Pa (standard)")
print(f"Calibration Pressure at 94 dB: {calibration_pressure:.6e} Pa")
print(f"Leq (dBA): {leq:.2f}")

# Optional: Play audio to verify
ipd.Audio(audio_file)  # Original audio
print("Cleaned Audio (first 10s):")
ipd.Audio(y_filtered[:int(10 * sr)], rate=sr)

In [ ]:
import librosa
import numpy as np
import IPython.display as ipd

# File path
audio_file = "data\\test.wav"

# Load audio
x_n, sr = librosa.load(audio_file, sr=None)  # x_n: normalized samples, sr: sampling rate

# Parameters for framing
frame_length = 2048  # Number of samples per frame
hop_length = 512    # Number of samples between successive frames

# Step 1: Frame the signal and compute the magnitude spectrum
# librosa.stft returns the Short-Time Fourier Transform
stft = np.abs(librosa.stft(x_n, n_fft=frame_length, hop_length=hop_length))

# Get frequencies corresponding to FFT bins
freqs = librosa.fft_frequencies(sr=sr, n_fft=frame_length)

# Step 2: Compute Spectral Centroid
# C[n] = (Σ_k=0^N-1 f_k * |X[k, n]|) / (Σ_k=0^N-1 |X[k, n]|)
centroid = librosa.feature.spectral_centroid(S=stft, sr=sr, hop_length=hop_length)

# Step 3: Compute Spectral Bandwidth
# B[n] = sqrt(Σ_k=0^N-1 (f_k - C[n])^2 * |X[k, n]|) / (Σ_k=0^N-1 |X[k, n]|)
bandwidth = librosa.feature.spectral_bandwidth(S=stft, sr=sr, hop_length=hop_length, centroid=centroid)

# Time axis for frames
frames = np.arange(centroid.shape[1])
times = librosa.frames_to_time(frames, sr=sr, hop_length=hop_length)

# Output results
print("Spectral Centroid (Hz) for each frame:")
for t, c in zip(times, centroid[0]):
    print(f"Time: {t:.3f}s, Centroid: {c:.2f} Hz")

print("\nSpectral Bandwidth (Hz) for each frame:")
for t, b in zip(times, bandwidth[0]):
    print(f"Time: {t:.3f}s, Bandwidth: {b:.2f} Hz")

# Optional: Play audio to verify
ipd.Audio(audio_file)

In [ ]:
import librosa
import numpy as np
import IPython.display as ipd
from scipy.signal import argrelextrema

# File path
audio_file = "data\\test.wav"
sampling_rate = 44100

# Load audio
x_n, sr = librosa.load(audio_file, sr=sampling_rate)  # x_n: normalized samples, sr: sampling rate

# Parameters for framing
frame_length = 2048  # Number of samples per frame
hop_length = 512    # Number of samples between successive frames

# Step 1: Compute magnitude spectrum
stft = np.abs(librosa.stft(x_n, n_fft=frame_length, hop_length=hop_length))

# Step 2: Extract timbre features
centroid = librosa.feature.spectral_centroid(S=stft, sr=sr, hop_length=hop_length)  # 2D array (1, n_frames)
bandwidth = librosa.feature.spectral_bandwidth(S=stft, sr=sr, hop_length=hop_length, centroid=centroid)  # 1D array

# Combine features into a single feature vector for change detection
features = np.vstack((centroid[0], bandwidth)).T  # Shape: (n_frames, 2)

# Step 3: Detect change points using a simple difference method
# Compute the difference between consecutive frames
feature_diff = np.linalg.norm(features[1:] - features[:-1], axis=1)  # Euclidean distance

# Smooth the difference to reduce noise
window_size = 5
smoothed_diff = np.convolve(feature_diff, np.ones(window_size)/window_size, mode='same')

# Identify local maxima in the difference (potential change points)
change_points_idx = argrelextrema(smoothed_diff, np.greater, order=3)[0]
change_points_time = librosa.frames_to_time(change_points_idx, sr=sr, hop_length=hop_length)

# Add start and end points
change_points_time = np.insert(change_points_time, 0, 0)
change_points_time = np.append(change_points_time, librosa.get_duration(y=x_n, sr=sr))

# Step 4: Segment the audio
segments = []
for i in range(len(change_points_time) - 1):
    start_time = change_points_time[i]
    end_time = change_points_time[i + 1]
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    segment = x_n[start_sample:end_sample]
    segments.append((start_time, end_time, segment))

# Output segment information
print("Detected Segments:")
for i, (start, end, _) in enumerate(segments):
    print(f"Segment {i + 1}: {start:.3f}s to {end:.3f}s")

# Optional: Save or play segments
for i, (start, end, segment) in enumerate(segments):
    output_file = f"segment_{i + 1}.wav"
    librosa.output.write_wav(output_file, segment, sr)
    print(f"Saved {output_file}")
    # ipd.Audio(segment, rate=sr)

# Optional: Play original audio to verify
ipd.Audio(audio_file)

In [ ]:
import librosa
import numpy as np
import IPython.display as ipd
from scipy.signal import argrelextrema
import soundfile as sf
import os

# File path
audio_file = "data\\test.wav"

# Load audio
x_n, sr = librosa.load(audio_file, sr=None)  # x_n: normalized samples, sr: sampling rate
total_duration = librosa.get_duration(y=x_n, sr=sr)

# Parameters for framing
frame_length = 2048  # Number of samples per frame
hop_length = 512    # Number of samples between successive frames

# Step 1: Compute magnitude spectrum
stft = np.abs(librosa.stft(x_n, n_fft=frame_length, hop_length=hop_length))

# Step 2: Extract timbre features
centroid = librosa.feature.spectral_centroid(S=stft, sr=sr, hop_length=hop_length)
bandwidth = librosa.feature.spectral_bandwidth(S=stft, sr=sr, hop_length=hop_length, centroid=centroid)

# Combine features into a single feature vector for change detection
features = np.vstack((centroid, bandwidth)).T  # Shape: (n_frames, 2)

# Step 3: Detect change points with threshold
# Compute the difference between consecutive frames
feature_diff = np.linalg.norm(features[1:] - features[:-1], axis=1)  # Euclidean distance

# Smooth the difference to reduce noise
window_size = 10  # Increased to smooth more aggressively
smoothed_diff = np.convolve(feature_diff, np.ones(window_size)/window_size, mode='same')

# Set a threshold (e.g., 95th percentile of smoothed differences)
threshold = np.percentile(smoothed_diff, 95)  # Adjust percentile (e.g., 90-98) based on audio
change_points_idx = np.where(smoothed_diff > threshold)[0]

# Convert to time
change_points_time = librosa.frames_to_time(change_points_idx, sr=sr, hop_length=hop_length)

# Add start and end points
change_points_time = np.insert(change_points_time, 0, 0)
change_points_time = np.append(change_points_time, total_duration)

# Step 4: Segment the audio with minimum 10-second duration
min_duration = 5.0  # Minimum segment duration in seconds
segments = []
current_start = change_points_time[0]
current_end = change_points_time[0]

for end_time in change_points_time[1:]:
    duration = end_time - current_start
    if duration >= min_duration or end_time == total_duration:
        current_end = end_time
        start_sample = int(current_start * sr)
        end_sample = min(int(current_end * sr), len(x_n))  # Ensure no out-of-bounds
        segment = x_n[start_sample:end_sample]
        segments.append((current_start, current_end, segment))
        current_start = current_end
    else:
        continue  # Merge with next segment

# If the last segment is less than 10s and not the end, extend to total duration
if len(segments) > 0 and (total_duration - segments[-1][1]) < min_duration:
    start_sample = int(segments[-1][1] * sr)
    end_sample = len(x_n)
    segment = x_n[start_sample:end_sample]
    segments[-1] = (segments[-1][0], total_duration, np.concatenate((segments[-1][2], segment)))

# Output folder (e.g., 'test' from 'test.wav')
folder_name = os.path.splitext(os.path.basename(audio_file))[0]  # Extract 'test' from 'test.wav'
output_folder = os.path.join(os.path.dirname(audio_file), folder_name)
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Output segment information
print("Detected Segments:")
for i, (start, end, _) in enumerate(segments):
    print(f"Segment {i + 1}: {start:.3f}s to {end:.3f}s (Duration: {end - start:.3f}s)")

# Save segments to the folder
for i, (start, end, segment) in enumerate(segments):
    output_file = os.path.join(output_folder, f"segment_{i + 1}.wav")
    sf.write(output_file, segment, sr)
    print(f"Saved {output_file}")
    # ipd.Audio(segment, rate=sr)

# Optional: Visualize smoothed difference to tune threshold
import matplotlib.pyplot as plt
times = librosa.frames_to_time(np.arange(len(smoothed_diff)), sr=sr, hop_length=hop_length)
plt.plot(times, smoothed_diff, label='Smoothed Difference')
plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold ({threshold:.2f})')
plt.xlabel("Time (s)")
plt.ylabel("Difference")
plt.title("Smoothed Feature Difference with Threshold")
plt.legend()
plt.show()

# Optional: Play original audio to verify
ipd.Audio(audio_file)

In [ ]:
import os
import numpy as np
import librosa
import pandas as pd

def apply_a_weighting(signal, sr):
    freqs = np.fft.rfftfreq(len(signal), d=1/sr)
    a_weights = np.ones_like(freqs)
    for i, f in enumerate(freqs):
        if f < 20 or f > 20000:
            a_weights[i] = 0  # Attenuate outside human hearing
        elif 1000 <= f <= 6000:
            a_weights[i] = 1.0  # Emphasize mid frequencies
        else:
            a_weights[i] = 0.8  # Slight attenuation elsewhere
    spectrum = np.fft.rfft(signal)
    spectrum_weighted = spectrum * a_weights
    signal_weighted = np.fft.irfft(spectrum_weighted, n=len(signal))
    return signal_weighted


folder_path = "data\\noiseData"  
output_csv = "leq_results.csv"


results = []


for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        
        
        try:
            y, sr = librosa.load(file_path, sr=None)  
            
            y_a_weighted = apply_a_weighting(y, sr)
            
            p_ref = 2e-5  
            mean_square = np.mean(y_a_weighted**2)
            leq = 10 * np.log10(mean_square / (p_ref**2))
            
            results.append([filename, leq])
            
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            results.append([filename, float('nan')])


if results:
    df = pd.DataFrame(results, columns=['Filename', 'Leq_dBA'])
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
else:
    print("No valid WAV files processed.")

In [ ]:
import os
import numpy as np
from scipy.io import wavfile
import pandas as pd

REF_PRESSURE = 20e-6
MIN_RMS_THRESHOLD = 1e-10

def calculate_leq(wav_data, sample_rate):
    rms = np.sqrt(np.mean(np.square(wav_data)))
    rms = max(rms, MIN_RMS_THRESHOLD)
    leq = 20 * np.log10(rms / REF_PRESSURE)
    return leq

folder_path = "data\\newwav" 
output_csv = "leq_results2.csv"


results = []


for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        
    
        try:
            sample_rate, data = wavfile.read(file_path)
            if data.ndim > 1:  
                data = np.mean(data, axis=1)
            
            
            if np.all(data == 0):
                print(f"Warning: {filename} is silent (all zeros). Skipping detailed analysis.")
                results.append([filename, float('nan'), float('nan'), float('nan')])
                continue
            
            
            max_amplitude = np.max(np.abs(data))
            if max_amplitude > 0 and max_amplitude < 1e-5:  
                data = data / max_amplitude * 32767  
            
            
            window_size = int(0.1 * sample_rate) 
            leq_values = []
            
            for i in range(0, len(data), window_size):
                segment = data[i:i + window_size]
                if len(segment) == window_size:
                    leq = calculate_leq(segment, sample_rate)
                    leq_values.append(leq)
            
            if leq_values:
                max_leq = np.max(leq_values)
                min_leq = np.min(leq_values)
                avg_leq = np.mean(leq_values)
                results.append([filename, max_leq, min_leq, avg_leq])
            else:
                results.append([filename, float('nan'), float('nan'), float('nan')])
                
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            results.append([filename, float('nan'), float('nan'), float('nan')])


if results:
    df = pd.DataFrame(results, columns=['Filename', 'Max_Leq_dB', 'Min_Leq_dB', 'Avg_Leq_dB'])
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
else:
    print("No valid WAV files processed.")

In [ ]:
import os
import numpy as np
import librosa
import pandas as pd

# Function to apply A-weighting (approximation)
def apply_a_weighting(signal, sr):
    freqs = np.fft.rfftfreq(len(signal), d=1/sr)
    a_weights = np.ones_like(freqs)
    for i, f in enumerate(freqs):
        if f < 20 or f > 20000:
            a_weights[i] = 0  # Attenuate outside human hearing
        elif 1000 <= f <= 6000:
            a_weights[i] = 1.0  # Emphasize mid frequencies
        else:
            a_weights[i] = 0.8  # Slight attenuation elsewhere
    spectrum = np.fft.rfft(signal)
    spectrum_weighted = spectrum * a_weights
    signal_weighted = np.fft.irfft(spectrum_weighted, n=len(signal))
    return signal_weighted

# Specify the folder containing WAV files
folder_path = "data\\noiseData" 
output_csv = "leq_result.csv"

# Lists to store results
results = []

# Loop through all WAV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        
        # Load audio
        try:
            y, sr = librosa.load(file_path, sr=None)  # y: audio samples, sr: sampling rate
            
            # Apply A-weighting
            y_a_weighted = apply_a_weighting(y, sr)
            
            # Calculate Leq for short segments (e.g., 0.1s windows)
            window_size = int(0.1 * sr)  # 0.1 seconds
            leq_values = []
            
            for i in range(0, len(y_a_weighted), window_size):
                segment = y_a_weighted[i:i + window_size]
                if len(segment) == window_size:
                    # Compute Leq for the segment
                    p_ref = 2e-5  # Reference pressure (Pa)
                    mean_square = np.mean(segment**2)  # Mean squared pressure
                    leq = 10 * np.log10(mean_square / (p_ref**2))
                    leq_values.append(leq)
            
            if leq_values:
                max_leq = np.max(leq_values)
                min_leq = np.min(leq_values)
                avg_leq = np.mean(leq_values)
                results.append([filename, max_leq, min_leq, avg_leq])
            else:
                results.append([filename, float('nan'), float('nan'), float('nan')])
                
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            results.append([filename, float('nan'), float('nan'), float('nan')])

# Create a DataFrame and save to CSV
if results:
    df = pd.DataFrame(results, columns=['Filename', 'Max_Leq_dBA', 'Min_Leq_dBA', 'Avg_Leq_dBA'])
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
else:
    print("No valid WAV files processed.")

In [ ]:
import os
import numpy as np
import librosa
import pandas as pd
from scipy import signal

def apply_a_weighting(signal, sr):
    """
    Apply A-weighting to a time-domain signal using librosa's A-weighting filter.
    
    Parameters:
        signal (np.ndarray): Input audio signal
        sr (int): Sampling rate in Hz
    
    Returns:
        np.ndarray: A-weighted signal
    """
    # Compute FFT frequencies
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=1/sr)
    
    # Get A-weighting values (in dB) for the frequencies
    a_weights_db = librosa.A_weighting(freqs)
    
    # Convert dB weights to linear scale
    a_weights = 10 ** (a_weights_db / 20.0)
    
    # Apply A-weighting in frequency domain
    spectrum = np.fft.rfft(signal)
    spectrum_weighted = spectrum * a_weights
    
    # Inverse FFT to get back to time domain
    signal_weighted = np.fft.irfft(spectrum_weighted, n=n)
    
    # Ensure output length matches input (handle any length mismatches)
    if len(signal_weighted) > len(signal):
        signal_weighted = signal_weighted[:len(signal)]
    elif len(signal_weighted) < len(signal):
        signal_weighted = np.pad(signal_weighted, (0, len(signal) - len(signal_weighted)), 'constant')
    
    return signal_weighted

def calculate_leq(signal, sr, p_ref=2e-5):
    """
    Calculate Leq (equivalent continuous sound level) in dB(A).
    
    Parameters:
        signal (np.ndarray): A-weighted audio signal
        sr (int): Sampling rate in Hz
        p_ref (float): Reference pressure (default: 2e-5 Pa)
    
    Returns:
        float: Leq value in dB(A)
    """
    # Compute mean square of the A-weighted signal
    mean_square = np.mean(signal**2)
    
    # Avoid log of zero or negative values
    if mean_square <= 0:
        return float('nan')
    
    # Calculate Leq in dB(A)
    leq = 10 * np.log10(mean_square / (p_ref**2))
    return leq

def process_audio_files(folder_path, output_csv):
    """
    Process WAV files in a folder, compute A-weighted Leq, and save results to CSV.
    
    Parameters:
        folder_path (str): Path to folder containing WAV files
        output_csv (str): Path to output CSV file
    """
    results = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(folder_path, filename)
            
            try:
                # Load audio file
                y, sr = librosa.load(file_path, sr=None, mono=True)
                
                # Apply A-weighting
                y_a_weighted = apply_a_weighting(y, sr)
                
                # Calculate Leq
                leq = calculate_leq(y_a_weighted, sr)
                
                results.append([filename, leq])
                print(f"Processed {filename}: Leq = {leq:.2f} dB(A)")
                
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                results.append([filename, float('nan')])
    
    # Save results to CSV
    if results:
        df = pd.DataFrame(results, columns=['Filename', 'Leq_dBA'])
        df.to_csv(output_csv, index=False)
        print(f"Results saved to {output_csv}")
    else:
        print("No valid WAV files processed.")

if __name__ == "__main__":
    folder_path = "data\\noiseData"  # Adjust path as needed
    output_csv = "new_leq_results.csv"
    
    # Ensure folder exists
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist.")
    else:
        process_audio_files(folder_path, output_csv)

In [ ]:
#LEQ STILL DIFFERES BY MUCH WITH THE MEASUREMENTS FROM THE SOUND LEVEL METER
import os
import numpy as np
import librosa
import pandas as pd
import scipy.signal as signal

def apply_a_weighting(signal, sr):
    """
    Apply A-weighting to a time-domain signal using librosa's A-weighting filter.
    
    Parameters:
        signal (np.ndarray): Input audio signal
        sr (int): Sampling rate in Hz
    
    Returns:
        np.ndarray: A-weighted signal
    """
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=1/sr)
    a_weights_db = librosa.A_weighting(freqs)
    a_weights = 10 ** (a_weights_db / 20.0)
    spectrum = np.fft.rfft(signal)
    spectrum_weighted = spectrum * a_weights
    signal_weighted = np.fft.irfft(spectrum_weighted, n=n)
    
    if len(signal_weighted) > len(signal):
        signal_weighted = signal_weighted[:len(signal)]
    elif len(signal_weighted) < len(signal):
        signal_weighted = np.pad(signal_weighted, (0, len(signal) - len(signal_weighted)), 'constant')
    
    return signal_weighted

def apply_time_weighting(signal, sr, time_weighting=None):
    """
    Apply time-weighting (fast or slow) to a signal.
    
    Parameters:
        signal (np.ndarray): Input signal
        sr (int): Sampling rate in Hz
        time_weighting (str): 'fast' (125 ms), 'slow' (1 s), or None
    
    Returns:
        np.ndarray: Time-weighted signal (squared for Leq calculation)
    """
    if time_weighting is None:
        return signal**2
    
    # Time constants
    tau = 0.125 if time_weighting.lower() == 'fast' else 1.0
    alpha = 1 - np.exp(-1/(sr * tau))
    
    # Exponential moving average (squared signal for Leq)
    b = [alpha]
    a = [1, -(1 - alpha)]
    return signal.lfilter(b, a, signal**2)

def calculate_leq(signal, sr, calibration_factor=1.0, time_weighting=None, p_ref=2e-5):
    """
    Calculate Leq in dB(A) with calibration and optional time-weighting.
    
    Parameters:
        signal (np.ndarray): Input audio signal
        sr (int): Sampling rate in Hz
        calibration_factor (float): Scaling factor to match SPL (default: 1.0)
        time_weighting (str): 'fast', 'slow', or None
        p_ref (float): Reference pressure (default: 2e-5 Pa)
    
    Returns:
        float: Leq value in dB(A)
    """
    # Apply calibration
    signal = signal * calibration_factor
    
    # Print signal diagnostics
    peak_amplitude = np.max(np.abs(signal))
    rms_unweighted = np.sqrt(np.mean(signal**2))
    print(f"  Peak amplitude: {peak_amplitude:.4f}")
    print(f"  RMS (unweighted): {20 * np.log10(rms_unweighted / p_ref):.2f} dB")
    
    # Apply A-weighting
    signal_weighted = apply_a_weighting(signal, sr)
    
    # Print A-weighted RMS
    rms_weighted = np.sqrt(np.mean(signal_weighted**2))
    print(f"  RMS (A-weighted): {20 * np.log10(rms_weighted / p_ref):.2f} dB")
    
    # Apply time-weighting to squared signal
    signal_squared = apply_time_weighting(signal_weighted, sr, time_weighting)
    
    # Compute mean square
    mean_square = np.mean(signal_squared)
    
    if mean_square <= 0:
        print("  Warning: Mean square is zero or negative, returning NaN")
        return float('nan')
    
    leq = 10 * np.log10(mean_square / (p_ref**2))
    return leq

def calibrate_signal(reference_file, sr, reference_spl=94.0):
    """
    Calculate calibration factor using a reference WAV file (e.g., 1 kHz tone at 94 dB SPL).
    
    Parameters:
        reference_file (str): Path to reference WAV file
        sr (int): Sampling rate in Hz
        reference_spl (float): Known SPL of reference signal (default: 94 dB)
    
    Returns:
        float: Calibration factor
    """
    y_ref, sr_ref = librosa.load(reference_file, sr=sr, mono=True)
    y_ref_weighted = apply_a_weighting(y_ref, sr_ref)
    mean_square = np.mean(y_ref_weighted**2)
    if mean_square <= 0:
        print("Warning: Reference signal mean square is zero, using default calibration factor")
        return 1.0
    measured_spl = 10 * np.log10(mean_square / (2e-5**2))
    calibration_factor = 10 ** ((reference_spl - measured_spl) / 20.0)
    return calibration_factor

def process_audio_files(folder_path, output_csv, calibration_file=None, time_weighting=None):
    """
    Process WAV files, compute A-weighted Leq, and save results to CSV.
    
    Parameters:
        folder_path (str): Path to folder containing WAV files
        output_csv (str): Path to output CSV file
        calibration_file (str): Path to reference WAV file for calibration (optional)
        time_weighting (str): 'fast', 'slow', or None
    """
    results = []
    
    # Calculate calibration factor if provided
    calibration_factor = 1.0
    if calibration_file:
        try:
            calibration_factor = calibrate_signal(calibration_file, sr=44100)
            print(f"Calibration factor: {calibration_factor:.4f}")
        except Exception as e:
            print(f"Error calibrating with {calibration_file}: {e}")
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(folder_path, filename)
            
            try:
                print(f"\nProcessing {filename}")
                y, sr = librosa.load(file_path, sr=None, mono=True)
                leq = calculate_leq(y, sr, calibration_factor, time_weighting)
                results.append([filename, leq])
                print(f"  Leq = {leq:.2f} dB(A)")
                
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                results.append([filename, float('nan')])
    
    if results:
        df = pd.DataFrame(results, columns=['Filename', 'Leq_dBA'])
        df.to_csv(output_csv, index=False)
        print(f"\nResults saved to {output_csv}")
    else:
        print("No valid WAV files processed.")

if __name__ == "__main__":
    folder_path = "data/noiseData"  # Adjust path
    output_csv = "leq_results.csv"
    calibration_file = None  # Set to path of reference WAV file (e.g., 94 dB SPL tone)
    time_weighting = 'slow'  # Options: 'fast', 'slow', or None
    
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist.")
    else:
        process_audio_files(folder_path, output_csv, calibration_file, time_weighting)

In [ ]:
import librosa
import numpy as np
from acoustics import Signal

# Load WAV file
y, sr = librosa.load('data\\noiseData\\Around Baba Dogo Rd(1°14_51_S 36°52_26_E).wav', sr=None, mono=True)

# Convert to pressure (assume 1 Pa for full scale)
p_scale = 1.0  # Adjust based on calibration
p = y * p_scale

# Define A-weighting function
def a_weighting(f):
    f = np.asarray(f)
    num = (12194**2) * (f**4)
    den = (f**2 + 20.6**2) * np.sqrt((f**2 + 107.7**2) * (f**2 + 737.9**2)) * (f**2 + 12194**2)
    weight = num / den
    weight_db = 20 * np.log10(weight) + 2.0  # Normalize at 1 kHz
    return weight_db

# Compute STFT
n_fft = 2048
hop_length = 512
stft = librosa.stft(p, n_fft=n_fft, hop_length=hop_length)
freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

# Apply A-weighting
weights_db = a_weighting(freqs)
weights = 10**(weights_db / 20.0)
stft_a_weighted = stft * weights[:, np.newaxis]

# Convert back to time domain using librosa.istft
p_a_weighted = librosa.istft(stft_a_weighted, hop_length=hop_length, length=len(p))

# Calculate Leq
p0 = 2e-5
mean_squared_pressure = np.mean(p_a_weighted**2)
leq = 10 * np.log10(mean_squared_pressure / (p0**2))

# Use acoustics' Leq method
signal_a_weighted = Signal(p_a_weighted, fs=sr)
leq_acoustics = signal_a_weighted.leq()

print(f"A-weighted Leq (manual): {leq:.2f} dB(A)")
print(f"A-weighted Leq (acoustics): {leq_acoustics:.2f} dB(A)")

In [ ]:
import librosa
import numpy as np
from acoustics import Signal
from scipy.signal import butter, sosfilt, sosfreqz
import scipy.signal as signal

# Load WAV file
y, sr = librosa.load('data\\noiseData\\Around Baba Dogo Rd(1°14_51_S 36°52_26_E).wav', sr=None, mono=True)

# Convert to pressure (adjust p_scale with calibration, e.g., 94 dB SPL = 1 Pa)
p_scale = 1.4
p = y * p_scale

# Design A-weighting filter (IEC 61672-1 compliant)
def a_weighting_filter(fs):
    """
    Design an IIR filter for A-weighting based on IEC 61672-1.
    Parameters:
        fs: Sampling frequency (Hz)
    Returns:
        sos: Second-order sections for filtering
    """
    # Pole and zero frequencies (Hz) from IEC 61672-1
    f1 = 20.6  # Low-frequency pole
    f2 = 107.7
    f3 = 737.9
    f4 = 12194.0  # High-frequency pole

    # Analog filter: zeros at 0 (double), f4 (double); poles at f1 (double), f2, f3, f4 (double)
    zeros = np.array([0, 0, -2*np.pi*f4, -2*np.pi*f4], dtype=np.float64)
    poles = np.array([-2*np.pi*f1, -2*np.pi*f1, -2*np.pi*f2, -2*np.pi*f3, -2*np.pi*f4, -2*np.pi*f4], dtype=np.float64)
    gain = (2*np.pi*f4)**4 / np.sqrt((2*np.pi*f2)*(2*np.pi*f3))

    try:
        # Convert to transfer function
        b, a = signal.zpk2tf(zeros, poles, gain)
        # Convert to SOS for stability
        sos = signal.tf2sos(b, a, analog=False)
        # Normalize gain at 1 kHz
        w, h = sosfreqz(sos, worN=2048, fs=fs)
        freq_1khz = np.argmin(np.abs(w - 1000))
        gain_1khz = np.abs(h[freq_1khz])
        sos = signal.tf2sos(b, a, analog=False, gain=1/gain_1khz)
    except Exception as e:
        print(f"Error designing filter: {e}")
        # Fallback to a simpler Butterworth approximation if needed
        sos = butter(4, [20, 20000], btype='band', fs=fs, output='sos')
        print("Using fallback Butterworth filter")

    return sos

# Create A-weighting filter
try:
    sos = a_weighting_filter(sr)
except Exception as e:
    print(f"Filter creation failed: {e}")
    raise

# Window parameters
window_size = int(sr * 10.0)  # 1-second windows
p0 = 2e-5  # Reference pressure (20 µPa)
leqs = []

# Process each 1-second window
for i in range(0, len(p), window_size):
    segment = p[i:i+window_size]
    if len(segment) < window_size // 2:  # Skip short segments
        continue
    # Apply A-weighting filter
    try:
        segment_a_weighted = sosfilt(sos, segment)
    except Exception as e:
        print(f"Error applying filter to segment {i//window_size}: {e}")
        continue
    # Calculate Leq for the segment
    mean_squared_pressure = np.mean(segment_a_weighted**2)
    if mean_squared_pressure <= 0:  # Avoid log of zero or negative
        continue
    leq = 10 * np.log10(mean_squared_pressure / (p0**2))
    if np.isfinite(leq):  # Check for valid Leq
        leqs.append(leq)

# Compute total Leq (logarithmic average)
if leqs:
    leq_total = 10 * np.log10(np.mean(10**(np.array(leqs)/10)))
    print(f"Total A-weighted Leq: {leq_total:.2f} dB(A)")
else:
    print("No valid windows processed")

# Optional: Compute Leq for the entire signal using acoustics
try:
    signal_full = Signal(p, fs=sr)
    p_a_weighted = sosfilt(sos, p)
    signal_a_weighted = Signal(p_a_weighted, fs=sr)
    leq_acoustics = signal_a_weighted.leq()
    print(f"A-weighted Leq (full signal, acoustics): {leq_acoustics:.2f} dB(A)")
except Exception as e:
    print(f"Error computing full-signal Leq: {e}")

Error designing filter: tf2sos() got an unexpected keyword argument 'gain'
Using fallback Butterworth filter
Total A-weighted Leq: 76.80 dB(A)
A-weighted Leq (full signal, acoustics): 76.80 dB(A)


In [1]:
import librosa
import numpy as np
from acoustics import Signal
from scipy.signal import butter, sosfilt, sosfreqz
import scipy.signal as signal

# Load WAV file
y, sr = librosa.load('data\\noiseData\\Around Baba Dogo Rd(1°14_51_S 36°52_26_E).wav', sr=44100, mono=True)

# Convert to pressure (adjust p_scale with calibration, e.g., 94 dB SPL = 1 Pa)
p_scale = 1.5  # Placeholder: calibrate using a known reference tone
p = y * p_scale

# Design A-weighting filter (IEC 61672-1 compliant)
def a_weighting_filter(fs):
    """
    Design an IIR filter for A-weighting based on IEC 61672-1.
    Parameters:
        fs: Sampling frequency (Hz)
    Returns:
        sos: Second-order sections for filtering
    """
    # Pole and zero frequencies (Hz) from IEC 61672-1
    f1 = 20.6  # Low-frequency pole
    f2 = 107.7
    f3 = 737.9
    f4 = 12194.0  # High-frequency pole

    # Analog filter: zeros at 0 (double), f4 (double); poles at f1 (double), f2, f3, f4 (double)
    zeros = np.array([0, 0, -2*np.pi*f4, -2*np.pi*f4], dtype=np.float64)
    poles = np.array([-2*np.pi*f1, -2*np.pi*f1, -2*np.pi*f2, -2*np.pi*f3, -2*np.pi*f4, -2*np.pi*f4], dtype=np.float64)
    gain = (2*np.pi*f4)**4 / np.sqrt((2*np.pi*f2)*(2*np.pi*f3))

    try:
        # Convert to transfer function
        b, a = signal.zpk2tf(zeros, poles, gain)
        # Convert to SOS for stability
        sos = signal.tf2sos(b, a, analog=False)
        # Normalize gain at 1 kHz
        w, h = sosfreqz(sos, worN=2048, fs=fs)
        freq_1khz = np.argmin(np.abs(w - 1000))
        gain_1khz = np.abs(h[freq_1khz])
        sos = signal.tf2sos(b, a, analog=False, gain=1/gain_1khz)
    except Exception as e:
        print(f"Error designing filter: {e}")
        # Fallback to a simpler Butterworth approximation if needed
        sos = butter(4, [20, 20000], btype='band', fs=fs, output='sos')
        print("Using fallback Butterworth filter")

    return sos

# Create A-weighting filter
try:
    sos = a_weighting_filter(sr)
except Exception as e:
    print(f"Filter creation failed: {e}")
    raise

# Window parameters
window_size = int(sr * 1.0)  # 1-second windows
p0 = 2e-5  # Reference pressure (20 µPa)
leqs = []

# Process each 1-second window
for i in range(0, len(p), window_size):
    segment = p[i:i+window_size]
    if len(segment) < window_size // 2:  # Skip short segments
        continue
    # Apply A-weighting filter
    try:
        segment_a_weighted = sosfilt(sos, segment)
    except Exception as e:
        print(f"Error applying filter to segment {i//window_size}: {e}")
        continue
    # Calculate Leq for the segment
    mean_squared_pressure = np.mean(segment_a_weighted**2)
    if mean_squared_pressure <= 0:  # Avoid log of zero or negative
        continue
    leq = 10 * np.log10(mean_squared_pressure / (p0**2))
    if np.isfinite(leq):  # Check for valid Leq
        leqs.append(leq)

# Compute total Leq (logarithmic average)
if leqs:
    leq_total = 10 * np.log10(np.mean(10**(np.array(leqs)/10)))
    print(f"Total A-weighted Leq: {leq_total:.2f} dB(A)")
else:
    print("No valid windows processed")

# Optional: Compute Leq for the entire signal using acoustics
try:
    signal_full = Signal(p, fs=sr)
    p_a_weighted = sosfilt(sos, p)
    signal_a_weighted = Signal(p_a_weighted, fs=sr)
    leq_acoustics = signal_a_weighted.leq()
    print(f"A-weighted Leq (full signal, acoustics): {leq_acoustics:.2f} dB(A)")
except Exception as e:
    print(f"Error computing full-signal Leq: {e}")

Error designing filter: tf2sos() got an unexpected keyword argument 'gain'
Using fallback Butterworth filter
Total A-weighted Leq: 77.40 dB(A)
A-weighted Leq (full signal, acoustics): 77.40 dB(A)
